# Rheological Flow around Confined Cylinder

Results published:
[Kikker, Kummer, Oberlack: A fully coupled high-order discontinuous Galerkin solver for viscoelastic fluid flow](https://onlinelibrary.wiley.com/doi/10.1002/fld.4950), 
see also at [tubiblio](http://tubiblio.ulb.tu-darmstadt.de/125045/).

### Preliminaries

This example can be found in the source code repository as as `ConfinedCylinder_ConvergenceStudy.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [1]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
//#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

The below script needs to be able to find the current output cell; this is an easy method to get it.

## Initialization tasks

Loading the `Rheology` namespace:

In [2]:
using BoSSS.Application.Rheology;

Initialization of the Workflow management; there `ConfinedCylinder_ConvergenceStudy` is the project name which is used for all comutations (aka. sessions):

In [3]:
BoSSSshell.WorkflowMgm.Init("ConfinedCylinder_ConvergenceStudy");

Project name is set to 'ConfinedCylinder_ConvergenceStudy'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [4]:
ExecutionQueues

index,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths
0,C:\BoSSStests,False,MSHPC-Gitrunner-HighPrio,dotnet,FDY\jenkinsci,DC2,[ fdygitrunner ],Highest,True,"[ { C:\BoSSStests == : LocalMountPath: C:\BoSSStests, PathAtRemote: }, { C:\Users\jenkinsci\ == : LocalMountPath: C:\Users\jenkinsci\, PathAtRemote: } ]"
1,C:\BoSSStests,False,MSHPC-Gitrunner-DefaultTest,dotnet,FDY\jenkinsci,DC2,[ fdygitrunner ],Normal,True,"[ { C:\BoSSStests == : LocalMountPath: C:\BoSSStests, PathAtRemote: } ]"
2,\\fdygitrunner\ValidationTests,False,MSHPC-AllNodes,dotnet,FDY\jenkinsci,DC2,"[ hpccluster, hpccluster2 ]",Normal,True,"[ { \\fdygitrunner\ValidationTests == : LocalMountPath: \\fdygitrunner\ValidationTests, PathAtRemote: } ]"


For this example (which is part of the BoSSS validation tests), a *default queue* is selected to run all jobs in the convergence study:

In [5]:
var myBatch = GetDefaultQueue();

Create a database (in a location that is accessible by the selected batch system):

In [6]:
var myDb = myBatch.CreateOrOpenCompatibleDatabase("Rheology_ConfinedCylinder");

Opening existing database '\\fdygitrunner\ValidationTests\Rheology_ConfinedCylinder'.


In [7]:
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

## Parameter set for Convergence Study

In [8]:
int[] pOrder = new int[] {1, 2, 3, 4};
int numberGrids = 4;

In [9]:
double[] WeissenbergS = new double[] { 0.0, 0.2 };

## Init grids and save to database

In [10]:
IGridInfo[] grids = new IGridInfo[numberGrids];
//GridCommons grid;

// Function to specify the boundary condition in dependence of the position X:
Func<double[], string> edgeTagFunc = delegate (double[] X) {
    double x = X[0];
    double y = X[1];
    //  if (!C.FixedStreamwisePeriodicBC)
    //  {
        if (Math.Abs(x - (-15)) < 1.0e-10)
            return "Velocity_inlet";
        if (Math.Abs(x - (15)) < 1.0e-10)
            return "Pressure_Outlet";
    //  }
    //  if (Math.Abs(y - (-2)) < 1.0e-10)
    //      return "Freeslip";
                        
        if (Math.Abs(y - (0)) < 1.0e-10)
            return "Freeslip";
                        
        if (Math.Abs(y - (+2)) < 1.0e-10)
            return "Wall_top";
                        
    //  if (-1.0 < y && y < 1.0 && -1.0 < x  && x < 1.0)
    //    return "Wall_cylinder";
                        
        if (0.0 < y && y < 1.0 && -1.0 < x  && x < 1.0)
            return "Wall_cylinder";
                        
        throw new ArgumentOutOfRangeException("at x = " + x + "and y = " +y);
    };

for (int k = 0; k < numberGrids; k++) { // loop over mesh resolutions...
    // import grid from Gmsh file:
    grids[k] = BoSSSshell.WorkflowMgm.ImportGrid(@"mesh_karman_OriginalBox_MEDIUM_" + k + "_half.msh", 
                                                 EdgeTagFunc:edgeTagFunc);
}


Grid Edge Tags changed.
An equivalent grid (ccb54ca3-d2c5-477d-b6d9-78cf77aca353) is already present in the database -- the grid will not be saved.

Grid Edge Tags changed.
An equivalent grid (7f65df88-0f1b-4fea-92db-339d001f444e) is already present in the database -- the grid will not be saved.

Grid Edge Tags changed.
An equivalent grid (8af57f3e-403f-4f20-8788-8f601012d208) is already present in the database -- the grid will not be saved.

Grid Edge Tags changed.
An equivalent grid (5b651e23-15aa-492a-b24a-9932043ff901) is already present in the database -- the grid will not be saved.


In [11]:
grids

#0: { Guid = ccb54ca3-d2c5-477d-b6d9-78cf77aca353; Name = ; Cell Count = 320; Dim = 2 }
#1: { Guid = 7f65df88-0f1b-4fea-92db-339d001f444e; Name = ; Cell Count = 1208; Dim = 2 }
#2: { Guid = 8af57f3e-403f-4f20-8788-8f601012d208; Name = ; Cell Count = 4252; Dim = 2 }
#3: { Guid = 5b651e23-15aa-492a-b24a-9932043ff901; Name = ; Cell Count = 14712; Dim = 2 }


## Setup of control objects for a solver runs

In [12]:
using BoSSS.Solution.AdvancedSolvers;

In [13]:
// Code for Initial conditions
class MyInitCode {

static string nl = System.Environment.NewLine;
public static string InitCode(double Reynolds, double Weissenberg, double beta) => 
"static class InitialValues { " + nl +
" " + nl +
"    static double u0                   = 1.5; // 0.375;// 0.66;// 3 / 2;   " + nl +
"    static double h                    = 4; " + nl +
$"    public static double beta          = {beta}; " + nl +
$"    static double Reynolds             = {Reynolds}; " + nl +
$"    static double Weissenberg          = {Weissenberg};  " + nl +
"    " + nl +
"    public static double VelocityXfunction(double[] X) { " + nl +
"        return u0 * (1  - (X[1] *  X[1])/h); " + nl +
"    } " + nl +
"    " + nl +
"    public static double VelocityYfunction(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"    " + nl +
"    public static double Pressurefunction(double[] X) { " + nl +
"        return u0 * 0.5 * Reynolds * (35 - X[0]); " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXXfunction_wi02(double[] X) { " + nl +
"        const double Weissenberg = 0.2; " + nl +
"        return 2 * Weissenberg * (1 - beta) * u0 * (-2 / h) * X[1] * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXXfunction_wi00(double[] X) { " + nl +
"        const double Weissenberg = 0.0; " + nl +
"        return 2 * Weissenberg * (1 - beta) * u0 * (-2 / h) * X[1] * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressXYfunction(double[] X) { " + nl +
"        return (1 - beta) * u0 * (-2 / h) * X[1]; " + nl +
"    } " + nl +
"    " + nl +
"    public static double StressYYfunction(double[] X) { " + nl +
"        return 0.0; " + nl +
"    } " + nl +
"}  " + nl;

}

In [14]:
//MyInitCode.InitCode(1,2)

In [15]:
List<RheologyControl> Controls = new List<RheologyControl>();

In [16]:
Controls.Clear();
double _beta = 0.59;
double _Reynolds = 1.0;
foreach(double Weissenberg in WeissenbergS) { // loop over different Weissenberg numbers...
foreach(int degree in pOrder) { //               loop over polynomial orders...
int elemInd = 0;
foreach(var grd in grids) { //                   loop over grid resolutions...

    //Database
    RheologyControl C = new RheologyControl();
    C.savetodb        = true;
    C.DbPath          = myDb.Path;
    C.SetGrid(grd);
    C.TracingNamespaces = "BoSSS,ilPSP";

    //SolverChooser
    C.NonLinearSolver.MaxSolverIterations  = 100;
    C.NonLinearSolver.MinSolverIterations  = 3;
    C.NonLinearSolver.ConvergenceCriterion = 0.0; // as accurate as possible, until no further reduction can be achieved.
    C.NonLinearSolver.SolverCode           = NonLinearSolverCode.Newton;
    C.LinearSolver.SolverCode = LinearSolverCode.classic_pardiso;

    //Timestepping
    C.TimesteppingMode   = AppControl._TimesteppingMode.Steady;
    C.Timestepper_Scheme = RheologyControl.TimesteppingScheme.ImplicitEuler;
    C.ObjectiveParam     = 1.0;

    //Configuration Shock capturing and body forces
    //C.UsePerssonSensor       = false;
    //C.SensorLimit            = 1e-4;
    C.AdaptiveMeshRefinement = false;
    C.RefinementLevel        = 10;
    //C.UseArtificialDiffusion = false;
    C.Bodyforces             = true;

    //Physical parameters
    C.beta                 = _beta;
    C.Reynolds             = _Reynolds;
    C.Weissenberg          = Weissenberg; //aim Weissenberg number!
    C.RaiseWeissenberg     = Weissenberg > 0.0;

    //Penalties
    C.ViscousPenaltyScaling = 1;
    C.Penalty2              = 1;
    C.Penalty1[0]           = 0.0;
    C.Penalty1[1]           = 0.0;
    C.PresPenalty2          = 1;  
    C.PresPenalty1[0]       = 0.0;
    C.PresPenalty1[1]       = 0.0;
    C.alpha                 = 1;  
    C.StressPenalty         = 1.0;

    //Create Fields
    C.SetDGdegree(degree);

    //Set initial values
    C.InitialValues.Add("VelocityX", new Formula("InitialValues.VelocityXfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.InitialValues.Add("VelocityY", new Formula("InitialValues.VelocityYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    if(Weissenberg == 0.0)
        C.InitialValues.Add("StressXX", new Formula("InitialValues.StressXXfunction_wi00", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else if(Weissenberg == 0.2)
        C.InitialValues.Add("StressXX", new Formula("InitialValues.StressXXfunction_wi02", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else
        throw new NotImplementedException("Missing Weissenberg = " + Weissenberg);    
    C.InitialValues.Add("StressXY", new Formula("InitialValues.StressXYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.InitialValues.Add("StressYY", new Formula("InitialValues.StressYYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));

    //Set Boundary Conditions
    //C.AddBoundaryValue("Wall_top", "VelocityX", Wall);
    //C.AddBoundaryValue("Wall_top", "VelocityY", Wall);
    //C.AddBoundaryValue("Wall_cylinder", "VelocityX", Wall);
    //C.AddBoundaryValue("Wall_cylinder", "VelocityY", Wall);

    C.AddBoundaryValue("Velocity_inlet", "VelocityX", new Formula("InitialValues.VelocityXfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.AddBoundaryValue("Velocity_inlet", "VelocityY", new Formula("InitialValues.VelocityYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    if(Weissenberg == 0.0)
        C.AddBoundaryValue("Velocity_inlet", "StressXX", new Formula("InitialValues.StressXXfunction_wi00", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else if(Weissenberg == 0.2)
        C.AddBoundaryValue("Velocity_inlet", "StressXX", new Formula("InitialValues.StressXXfunction_wi02", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    else
        throw new NotImplementedException("Missing Weissenberg = " + Weissenberg);
    C.AddBoundaryValue("Velocity_inlet", "StressXY", new Formula("InitialValues.StressXYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta)));
    C.AddBoundaryValue("Velocity_inlet", "StressYY", new Formula("InitialValues.StressYYfunction", false, MyInitCode.InitCode(1, Weissenberg, _beta))); 
            
    //Save Session and next...
    C.SessionName = $"ConfinedCylinder_ConvergenceStudy_p{degree}_meshNo{elemInd}_Wi{Weissenberg}_half";
    Controls.Add(C);
    Console.WriteLine("Created control: " + C.SessionName);
    elemInd++;
}
}
}

Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo1_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo2_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p1_meshNo3_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo1_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo2_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p2_meshNo3_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo1_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo2_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p3_meshNo3_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p4_meshNo0_Wi0_half
Created control: ConfinedCylinder_ConvergenceStudy_p4_meshNo1_Wi0_half
Create

## Launch Jobs

In [17]:
Controls.Count

32

In [18]:
foreach(var ctrl in Controls) {
    var oneJob              = ctrl.CreateJob();
    oneJob.NumberOfMPIProcs = 1;
    oneJob.Activate(myBatch); 
}

Info: Found successful session "ConfinedCylinder_ConvergenceStudy	ConfinedCylinder_ConvergenceStudy_p1_meshNo0_Wi0_half	10/22/2021 07:39:38	d78ed834..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
Info: Found successful session "ConfinedCylinder_ConvergenceStudy	ConfinedCylinder_ConvergenceStudy_p1_meshNo1_Wi0_half	10/22/2021 07:39:45	70d1ae1c..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
Info: Found successful session "ConfinedCylinder_ConvergenceStudy	ConfinedCylinder_ConvergenceStudy_p1_meshNo2_Wi0_half	10/22/2021 07:39:54	a538c206..." -- job is marked as successful, no further action.
No submission, because job status is: FinishedSuccessful
Info: Found successful session "ConfinedCylinder_ConvergenceStudy	ConfinedCylinder_ConvergenceStudy_p1_meshNo3_Wi0_half	10/22/2021 07:40:04	eba91046..." -- job is marked as successful, no further action.
No submission

NullReferenceException during job status evaluation: Object reference not set to an instance of an object.
NullReferenceException during job status evaluation: Object reference not set to an instance of an object.


Unhandled exception: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Application.BoSSSpad.Job.<GetAllUnkonwnExistingDeployDirectories>g__DirMatch|38_0(DirectoryInfo dir1, DirectoryInfo dir2) in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 759
   at BoSSS.Application.BoSSSpad.Job.<GetAllUnkonwnExistingDeployDirectories>g__DirIsKnown|38_1(DirectoryInfo dir) in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 766
   at BoSSS.Application.BoSSSpad.Job.GetAllUnkonwnExistingDeployDirectories() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 789
   at BoSSS.Application.BoSSSpad.Job.UpdateDeployments() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 883
   at BoSSS.Application.BoSSSpad.Job.get_AllDeployments() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\Bo

In [19]:
// wait for all jobs to finish (up to 5 days, check every 5 minutes)
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*5), PollingIntervallSeconds:(60*5));

Unhandled exception: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Application.BoSSSpad.Job.<GetAllUnkonwnExistingDeployDirectories>g__DirMatch|38_0(DirectoryInfo dir1, DirectoryInfo dir2) in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 759
   at BoSSS.Application.BoSSSpad.Job.<GetAllUnkonwnExistingDeployDirectories>g__DirIsKnown|38_1(DirectoryInfo dir) in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 766
   at BoSSS.Application.BoSSSpad.Job.GetAllUnkonwnExistingDeployDirectories() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 789
   at BoSSS.Application.BoSSSpad.Job.UpdateDeployments() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 883
   at BoSSS.Application.BoSSSpad.Job.get_AllDeployments() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\Bo

In [20]:
// detect failed Jobs in the job management
var suspects = BoSSSshell.WorkflowMgm.AllJobs.Select(kv => kv.Value)
    .Where(job => job.LatestSession.Tags.Contains(SessionInfo.NOT_TERMINATED_TAG)
                  || job.LatestSession.Tags.Contains(SessionInfo.SOLVER_ERROR)).ToArray();
suspects

Unhandled exception: System.NullReferenceException: Object reference not set to an instance of an object.
   at BoSSS.Application.BoSSSpad.Job.<GetAllUnkonwnExistingDeployDirectories>g__DirMatch|38_0(DirectoryInfo dir1, DirectoryInfo dir2) in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 759
   at BoSSS.Application.BoSSSpad.Job.<GetAllUnkonwnExistingDeployDirectories>g__DirIsKnown|38_1(DirectoryInfo dir) in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 766
   at BoSSS.Application.BoSSSpad.Job.GetAllUnkonwnExistingDeployDirectories() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 789
   at BoSSS.Application.BoSSSpad.Job.UpdateDeployments() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\BoSSSpad\Job.cs:line 883
   at BoSSS.Application.BoSSSpad.Job.get_AllDeployments() in C:\Users\jenkinsci\Documents\BoSSS-NET5\public\src\L4-application\Bo

In [21]:
NUnit.Framework.Assert.IsTrue(suspects.Count() <= 0, $"{suspects.Count()} Failed Jobs of {BoSSSshell.WorkflowMgm.AllJobs.Count()} in total.");

Unhandled exception: System.ArgumentNullException: Value cannot be null. (Parameter 'source')
   at System.Linq.ThrowHelper.ThrowArgumentNullException(ExceptionArgument argument)
   at System.Linq.Enumerable.Count[TSource](IEnumerable`1 source)
   at Submission#23.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

### Inspect the output of some arbitrary job:

In [22]:
BoSSSshell.WorkflowMgm.AllJobs.First().Value.Stdout

Session ID: d78ed834-571a-4140-ad49-5cdd6cbca02a, DB path: '\\fdygitrunner\ValidationTests\Rheology_ConfinedCylinder'.
configuring Weissenberg number: 0e+00
setting Weissenberg Number to 0
Total number of cells:    320
Total number of DOFs:     5120
Instationary solve, timestep #1.0, dt = 1.7976931348623158E+304 ...
 #Line, #Time, #Iter	L2Norm momX         L2Norm momY         L2Norm div          L2Norm constitutiveXXL2Norm constitutiveXYL2Norm constitutiveYYL2Norm Total        
     1,     1,     1	8.004800E+002       1.716621E-002       4.332954E+000       3.619002E+000       1.597659E+000       0.000000E+000       8.005015E+002       
     2,     1,     2	1.248636E+001       2.612047E+000       9.891723E-015       1.069542E-014       1.225540E-014       3.888714E-015       1.275665E+001       
     3,     1,     3	2.810957E-003       2.169423E-003       2.488120E-015       1.049136E-014       1.230991E-014       3.981671E-015       3.550757E-003       
     4,     1,     4	7.131471E-